# Compute Results of Analysis

Combine SIFT, resnet, swin, vgg and vit features with basic distance analysis, query expansion and diffusion.

____
## Imports and Constants

In [1]:
import numpy as np
import pandas as pd
import os
import sklearn.metrics.pairwise
import my_eval
import query

###########################################

NOTEBOOK_DIR = "/home/sean/Code/Pawsey/4. Clean"

_____
## Load Data

In [2]:
## Load features

data = {}

for descriptor in os.listdir("./data"):

    if descriptor == "names":
        data[descriptor] = {"ox" : {}, "par" : {}}
        for fname in os.listdir("./data/" + descriptor):
            split_name = fname[:-4].split("-")
            dataset = split_name[0]
            if fname.endswith("y-names.npy"):
                data[descriptor][dataset]["y"] = np.load("./data/{}/{}".format(descriptor, fname))
            else:
                difficulty = split_name[2]
                data[descriptor][dataset][difficulty] = np.load("./data/{}/{}".format(descriptor, fname))

    else:
        data[descriptor] = {"ox" : {"E" : {}, "M" : {}, "H" :{}},
                            "par" : {"E" : {}, "M" : {}, "H" :{}}}
        for fname in os.listdir("./data/" + descriptor):
            split_name = fname[:-4].split("-")
            if len(split_name) == 3:
                pass
            else:
                (_, xy, dataset, difficulty) = split_name
                data[descriptor][dataset][difficulty][xy] = np.load("./data/{}/{}".format(descriptor, fname))
    
    print("Loaded " + descriptor)

data["swin"]["ox"]["E"]["x"]

Loaded swin
Loaded vgg
Loaded resnet
Loaded names
Loaded vit


array([[ 0.4381919 , -1.1369115 , -0.49100572, ..., -0.27456677,
         0.38102797, -0.30554023],
       [-0.19804995,  0.02098738,  0.52111053, ...,  0.44540596,
         0.8620084 ,  0.18907186],
       [ 1.0216093 , -0.06300209, -0.06569103, ...,  0.02202551,
        -0.32440802,  0.3858102 ],
       ...,
       [ 0.74518114, -0.9655011 , -0.55623275, ..., -0.39560622,
         0.3983633 , -0.4672271 ],
       [ 0.4493655 , -0.97439206, -0.61376625, ..., -0.19914342,
         0.27447924, -0.3482531 ],
       [ 0.04259995,  0.09633142,  0.65417933, ...,  0.5438953 ,
         0.53027916,  0.03832415]], dtype=float32)

In [7]:
q = data["swin"]["ox"]["E"]["y"]
t = data["swin"]["ox"]["E"]["x"]
q_n = data["names"]["ox"]["y"]
t_n = data["names"]["ox"]["E"]

ranks = query.return_ranks('expanded query', q, t)
len(ranks)

AttributeError: 'numpy.ndarray' object has no attribute 'loc'

In [4]:
ranks

array([[239, 259, 276, ..., 184, 141, 515],
       [434, 507,  79, ..., 184, 141, 515],
       [213, 216, 462, ..., 184, 141, 515],
       ...,
       [213, 462,  89, ..., 184, 141, 515],
       [241,  37, 134, ..., 184, 141, 515],
       [434, 507,  79, ..., 184, 141, 515]])

In [5]:
my_eval.evaluate(ranks, q_n, t_n)

(0.946203674510284,
 {1: 0.9285714285714286, 5: 0.8000000000000002, 10: 0.7357142857142857})

_____
## Evaluate Methods

In [4]:
feature_types, datasets, difficulties, query_types, m_APs = [], [], [], [], []

for feature_type in ["swin", "vgg", "resnet", "vit"]:
    for dataset in ["ox", "par"]:
        for difficulty in ["E", "M", "H"]:
            queries = data[feature_type][dataset][difficulty]["y"]
            gallery = data[feature_type][dataset][difficulty]["x"]
            query_names = data["names"][dataset]["y"]
            gallery_names = data["names"][dataset][difficulty]
            
            for i in range(3):
                feature_type.append()

            ranks = query.return_ranks('basic', queries, gallery, metric_function = sklearn.metrics.pairwise.cosine_distances)
            mAP, precisions = my_eval.evaluate(ranks, query_names, gallery_names)
            print(mAP)

0.9261358175094431
0.9073505021849247
0.896813510052975
0.9998563818817134
0.9961624958766858
0.9933816229795974
0.7702650185187713
0.6936577906313003
0.6793737825003611
0.9999790002913993
0.9995414729807018
0.9987283115102363
0.8433871242193302
0.8017895131082906
0.7833786196998352
0.9999729842836189
0.9994494387571072
0.9981388112203007
0.9117628667549909
0.8514689116597296
0.8266221264342415
0.9999949642911687
0.9995945341170379
0.9983795259835458
